In [24]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
from scipy.stats import norm

import pandas as pd
from pynverse import inversefunc

In [85]:
r0 = 0 # rate of intrests
b = 0.1 # drift term of Black sholes
sigma = 0.3

# Calculate Euroupe call price
def call_price(K,S0,sigma,r=r0,T=1.0):
    v = sigma* np.sqrt(T)
    d1 = (np.log(S0/K)+r*T+v**2/2)/v
    d2 = d1-v
    return norm.cdf(d1)*S0-norm.cdf(d2)*K*np.exp(-r*T)

def delta_call(t,K,S,sigma=sigma,r=r0,T=1.0):
    v = sigma* np.sqrt(T-t)
    d1 = (np.log(S/K)+r*(T-t)+v**2/2)/v
    
    return norm.cdf(d1)

def Gamma_call(t,K,S,sigma=sigma,r=r0,T=1.0) :
    v = sigma* np.sqrt(T-t)
    d1 = (np.log(S/K)+r*(T-t)+v**2/2)/v
    
    return norm.pdf(d1)/(S*v)


def Yt(M=10000,S0=100,r=b,T=1.0,sigma=sigma, N = 80001):
    t = T/(N-1)
    # initialisation of stock price
    yt = np.exp(t*(np.ones((M,N)).cumsum(axis=1)-1)*(r-0.5*sigma**2))*S0
    Wt = npr.randn(M,N)*np.sqrt(t)
    Wt[:,0]=0
    Wt = Wt.cumsum(axis=1)
    yt = yt*np.exp(sigma*Wt)
    return yt

In [274]:
epsilon =np.concatenate([np.arange(10,20,2)/10,np.arange(20,44,4)/10])

In [25]:
# Define the function we need to inverse in the calculation of Pt
fun_L = (lambda x: x*np.exp(x))
L_inverse = inversefunc(fun_L)

In [35]:
def Pt_and_dPt(t,T=1.0,mu=1,b=0.1,sigma=0.3):
    rho = b/sigma
    tmp = 0.5*np.exp(rho**2*(T-t)+0.5)
    Pt = mu/L_inverse(tmp)
    dPt = rho**2*(Pt**2)/(Pt+mu)
    return Pt,dPt

In [49]:
p = (np.ones(10).cumsum(axis=0)-1)
p=np.apply_along_axis((lambda x: Pt_and_dPt(x)),0,p)

In [251]:
def yt_st(M=10000,S0=100,r=b,T=1.0,sigma=sigma, N = 80001,mu=1):
    t = T/(N-1)
     # initialisation of stock price
    yt = np.exp(t*(np.ones((M,N)).cumsum(axis=1)-1)*(r-0.5*sigma**2))*S0
    
    dWt = npr.randn(M,N)*np.sqrt(t)
    dWt[:,0]=0
    Wt = dWt.cumsum(axis=1)
    yt = yt*np.exp(sigma*Wt)
    
    # Caculate pt and dpt
    pt_dpt = t*(np.ones(N).cumsum(axis=0)-1)
    pt_dpt = np.apply_along_axis((lambda x: Pt_and_dPt(x,mu=mu)),0,pt_dpt)
    pt = pt_dpt[0]
    dpt = pt_dpt[1]
    
    dpt_over_pt = dpt/pt
    rho = b/sigma
    # We want to caculate tilde(z_star) in paper
    zt = np.zeros((M,N))
    zt[:,0]=-1.0/(2*mu)
    for i in range(1,N):
        zt[:,i]=zt[:,i-1]+zt[:,i-1]*(-dpt_over_pt[i-1])*(t+dWt[:,i]/rho)
    
    st = -3/b*dpt_over_pt*(zt/yt)
    
    return yt,st,zt
    

In [ ]:
yt,st,zt = yt_st(M=10000)

In [ ]:
plt.plot(yt[0]);

In [ ]:
plt.plot(st[0]);

In [ ]:
def Jiatu(Yt,st,epsilon,K,d=0,r=r0,T=1.0,sigma=sigma): # here d is delta in the paper
    M,N = Yt.shape
    dt = T/(N-1)

    if d==0:
        l_d_t = st*epsilon
        l_u_t = np.zeros((M,N))
    else:
        l_d_t = (np.sqrt((st**2)/4+(6*d)/sigma**2)+st/2)*epsilon
        l_u_t = (np.sqrt((st**2)/4+(6*d)/sigma**2)-st/2)*epsilon
    
    
    hedge_bs = np.ones(M)*call_price(K,Yt[0,0],sigma,r,T) # Hedging by Black-sholes
    hedge_jia = np.ones(M)*call_price(K,Yt[0,0],sigma,r,T) # Hedging by Jiatu
    
    delta_bs = delta_call(0,K,Yt[:,0])
    delta_jia = delta_call(0,K,Yt[:,0])
    
    last_price_bs = Yt[:,0]
    last_price_jia = Yt[:,0]
    
    NR_jia = np.zeros(M)
    
    
    for i in range(1,N-1):
        # Update standard BS hedging portfolio
        hedge_bs+=delta_bs*(Yt[:,i]-last_price_bs)
        delta_bs = delta_call(dt*i,K,Yt[:,i])
        last_price_bs = Yt[:,i]
        # Update Massaki portfolio
        #tmp = Yt[:,i]>=(last_price_jia+l_u_t[:,i])
        tmp =np.logical_or((delta_bs>=delta_jia+l_u_t[:,i]),(delta_bs<=delta_jia-l_d_t[:,i]))
        
        hedge_jia+=tmp*(Yt[:,i]-last_price_jia)*delta_jia
        delta_jia =tmp*delta_call(dt*i,K,Yt[:,i])+(1-tmp)*delta_jia
        last_price_jia=tmp*Yt[:,i]+(1-tmp)*last_price_jia
        
        #print(sum(gamma_ma))
        NR_jia+=tmp
     
    hedge_bs+=delta_bs*(Yt[:,-1]-last_price_bs)
    hedge_jia+=delta_jia*(Yt[:,-1]-last_price_jia)
    
    return hedge_jia-hedge_bs,NR_jia

In [ ]:
z,nr = Jiatu(yt,st,epsilon[0],100)

In [ ]:
EZn=[]
EZn2=[]

for e in epsilon:
    z,nr = Jiatu(yt,st,e,100)
    EZn.append(z.mean())
    EZn2.append((z*z).mean())
    print("epsilon {} is done..".format(e))

plt.plot(EZn,EZn2)

In [ ]:
plt.scatter(EZn,EZn2)